In [226]:
import pandas as pd
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import datetime
from dotenv import load_dotenv
import os

In [227]:
download_path = os.path.join(os.path.expanduser('~'), 'Downloads')

In [228]:
file_path = rf"{download_path}\Base Consolidada 2025.xlsx"

In [ ]:
email_remetente = "projetos@polijunior.com.br"
load_dotenv(dotenv_path='.venv/.env')
senha = 'projetos.pj'

In [229]:
error_log = []

In [230]:
def send_email(remetente, destinatario, assunto, corpo, senha):
        try:
                # Criação da mensagem
                msg = MIMEMultipart()
                msg['From'] = remetente
                msg['To'] = destinatario
                msg['Subject'] = assunto

                # Corpo do e-mail
                msg.attach(MIMEText(corpo, 'html', 'utf-8'))

                # Conectar ao servidor SMTP do Gmail
                server = smtplib.SMTP('smtp.gmail.com', 587)
                server.starttls()  # Encriptação TLS

                # Login no servidor
                server.login(remetente, senha)

                # Envio do e-mail
                text = msg.as_string()
                server.sendmail(remetente, destinatario, text)

                # Encerrar a conexão
                server.quit()

                print(f"E-mail enviado com sucesso para {destinatario}")
        except Exception as e:
                error_log.append(f"Falha ao enviar e-mail para {destinatario}: {e}")

# Base Consolidada

In [ ]:
base_consolidada = pd.read_excel(file_path, sheet_name='Base')
forms_csat = pd.read_excel(file_path, sheet_name='CSAT Forms')

In [ ]:
# Filtragem de nulls
base_consolidada = base_consolidada[base_consolidada['Projeto'].notna()].reset_index(drop=True)

In [ ]:
base_consolidada_exec = base_consolidada[base_consolidada['Status'] == 'Executando'].copy()
validadores = base_consolidade_exec[base_consolidade_exec_atual['Validador/Coordenador'].notna()]['Validador/Coordenador'].unique()

count = 0
for validador in validadores:
    corpo_final = ''
    projetos_validados_df = base_consolidade_exec[base_consolidade_exec['Validador/Coordenador'] == validador]
    if validador == 'enzo.giannoni':
        for index, row in projetos_validados_df.iterrows():
            projeto = row['Projeto']
            portifolio = row['Portfólio']
            fim_previsto = row['Fim Previsto']
            quantidade_de_semanas_off = row['Semanas OFF']
            csat_status = row['CSAT coletado?']
            corpo_padrao = f'''
            <!DOCTYPE html>
            <html lang="pt">
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1.0">
            </head>
            <corpo>
                <h1>Detalhes: {projeto}</h1>
                <p><strong>Portfólio:</strong> {portifolio}</p>
                <p><strong>Fim previsto:</strong> {fim_previsto}</p>
                <p><strong>Quantidade de Semanas Off:</strong> {quantidade_de_semanas_off}</p>
            </corpo>
            </html>
            '''
            if csat_status:
                csat_df = forms_csat[forms_csat['Projeto'] == projeto]
                try:
                    csat_comunicacao = csat_df['As atualizações (Comunicação) da equipe a respeito do andamento do projeto tem sido satisfatórias?'].iloc[0]
                    csat_qualidade = csat_df['Como você avaliaria a sua satisfação com a qualidade da entrega da Poli Júnior até então? '].iloc[0]
                    csat_satisfacao = csat_df['Como você avaliaria a sua satisfação com o trabalho da Poli Júnior como um todo?'].iloc[0]
                    feedback = csat_df['Espaço para possíveis feedbacks qualitativos:'].iloc[0]
                except IndexError:
                    csat_comunicacao = None
                    csat_qualidade = None
                    csat_satisfacao = None
                if csat_comunicacao and csat_qualidade and csat_satisfacao:
                    corpo_csat = f''' 
                    <h2> CSAT </h2>
                    <p><strong>Nota de Comunicação:</strong> {csat_comunicacao}</p>
                    <p><strong>Nota de Entrega:</strong> {csat_qualidade}</p>
                    <p><strong>Nota do Trabalho:</strong> {csat_satisfacao}</p>
                    <p><strong>Feedbacks:</strong> {feedback}</p>'''

                    corpo_padrao = corpo_padrao + corpo_csat
            data_atual = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
            if data_atual > fim_previsto:
                corpo_msg_atrasado = '''
                <corpo>
                    <p style="color: red; font-weight: bold;">O PROJETO ESTÁ ATRASADO!</p>
                </corpo>
                '''
                corpo_padrao = corpo_padrao + corpo_msg_atrasado

            corpo_final = corpo_final + corpo_padrao
    destinatario = validador + '@polijunior.com.br'
    send_email(remetente=send_email, destinatario=destinatario, destinatario=destinatario, )
    print(validador, projeto, portifolio, fim_previsto, quantidade_de_semanas_off, csat_status, csat_comunicacao, csat_qualidade, csat_satisfacao)
    count += 1
    if count == 20:
        break